## Setup

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import os
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

In [ ]:
BATCH_SIZE = 32
img_height = 224
img_width = 224
IMAGE_SIZE = (img_width, img_height)


### Dataset

Usando dataset de gatos e cachorros:

In [ ]:
data_dir = tf.keras.utils.get_file(
  'a',
  'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
  cache_dir='/content',
  extract=True,
  archive_format='zip')

data_dir = '/content/datasets/cats_and_dogs_filtered'

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    data_dir + '/train', subset="training", shuffle=True, **dataflow_kwargs)

valid_generator = valid_datagen.flow_from_directory(
    data_dir + '/train', subset="validation", shuffle=False, **dataflow_kwargs)

In [ ]:
class_names = np.array([k for k in train_generator.class_indices.keys()])
print(class_names)

In [ ]:
for image_batch, labels_batch in train_generator:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Extraindo modelo

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(img_height, img_width, 3), trainable=False)

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

### Ajustando feature extractor layer

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(train_generator.num_classes,
                        kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

### Treinando o modelo

Compilando modelo ...

In [ ]:
base_learning_rate = 0.005
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

Treinando o modelo com o dataset de gatos e cachorros para somente 10 epocas

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
history = model.fit(
    train_generator,
    epochs=10, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps)

Analisando o history do treinamento

In [ ]:
print(history.history)

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

### Chegando predições para um lote de imagens qualquer

Aplicamos predict em image_batch e capturamos a maior probabilidade com a função argmax:

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

Plot dos resultados

In [ ]:
plt.figure(figsize=(15,13))
plt.subplots_adjust(hspace=0.5)
for n in range(32):
  plt.subplot(6,6,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

### Executando predições para imagens aleatorias

Baixando uma imagem simples de gato e cachorro



In [ ]:
my_cat = tf.keras.utils.get_file('/content/sample_data/my_cat.png',
                                  'https://ichef.bbci.co.uk/news/976/cpsprodpb/12A9B/production/_111434467_gettyimages-1143489763.jpg')
my_cat = Image.open(my_cat).resize(IMAGE_SIZE)
my_cat

In [ ]:
my_dog = tf.keras.utils.get_file('/content/sample_data/my_dog.png',
                                  'https://www.sciencemag.org/sites/default/files/styles/article_main_image_-_1280w__no_aspect_/public/dogs_1280p_0.jpg?itok=6jQzdNB8')
my_dog = Image.open(my_dog).resize(IMAGE_SIZE)
my_dog

Normalizamos as imagens para o intervalo 0-1

In [ ]:
my_cat = np.array(my_cat)/255.0
my_dog = np.array(my_dog)/255.0

Preditando o gato

In [ ]:
new_prediction_1 = model.predict(my_cat[np.newaxis, ...])
print(new_prediction_1)

predicted_id = np.argmax(new_prediction_1, axis=-1)
predicted_label = class_names[predicted_id]
print(predicted_label)

plt.imshow(my_cat)
plt.title(predicted_label[0])

Preditando o cachorro

In [ ]:
new_prediction_2 = model.predict(my_dog[np.newaxis, ...])
print(new_prediction_2)

predicted_id = np.argmax(new_prediction_2, axis=-1)
predicted_label = class_names[predicted_id]
print(predicted_label)

plt.imshow(my_dog)
plt.title(predicted_label[0])